In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'

In [2]:
!export HF_HOME=/shared/data3/pk36/.cache

In [3]:
from model_definitions import bertEncode
from taxonomy import Node, Taxonomy
import subprocess
import pickle as pk
from tqdm import tqdm
import numpy as np
from collections import deque
import json
from collections import defaultdict, Counter
from itertools import compress
from nltk.corpus import stopwords
import time
import torch
from scipy import stats
import math
from transformers import BertModel, BertTokenizer
from vllm import LLM, SamplingParams
#from model_definitions import llama_8b_model, sentence_model, promptLlama, constructPrompt, bertEncode
from utils import *
from prompts import *
from main import commonSenseEnrich, computeClassEmb

WARNING 09-23 05:48:00 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 09-23 05:48:00 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 09-23 05:48:00 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_p

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-23 05:48:35 model_runner.py:732] Loading model weights took 14.9888 GB
INFO 09-23 05:48:36 gpu_executor.py:102] # GPU blocks: 9057, # CPU blocks: 2048
INFO 09-23 05:48:38 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 05:48:38 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 05:48:46 model_runner.py:1225] Graph capturing finished in 8 secs.


/home/pk36/Comparative-Summarization/taxoadapt/env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at /home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
class Args:
    def __init__(self):
        self.dataset = "gen_kgc"
        self.data_dir = f"datasets/{self.dataset}/"
        self.internal = f"{self.dataset}.txt"
        self.external = f"{self.dataset}_external.txt"
        self.groundtruth = "groundtruth.txt"
        
        self.length = 512
        self.dim = 768

        self.iters = 4
        self.model = "bert_full_ft"
        self.override = True
        self.max_depth = 5

args = Args()

**Reading in input**

In [27]:
# create taxonomy from input
taxo = Taxonomy(args.data_dir)

taxo_dict = taxo.toDict(cur_node=taxo.root)
with open(f'datasets/{args.dataset}/initial.json', 'w') as fp:
    json.dump(taxo_dict, fp, indent=4)
dict_str = json.dumps(taxo_dict, indent=4)

**Common-sense enrichment**

In [28]:
enrich_start = time.time()
prompts, outputs, all_common_phrases, all_common_sentences = commonSenseEnrich(taxo.root, dict_str, True)
# prompts, output_dict = commonSenseEnrich(taxo.root, dict_str, True)
enrich_end = time.time()

if all_common_phrases:
    # update vocabulary/embeddings
    taxo.updateVocab(all_common_phrases, 'phrases')
    taxo.updateVocab(all_common_sentences, 'sentences')
    print(f"Time taken: {enrich_end - enrich_start} seconds ({(enrich_end - enrich_start)/60} minutes)")

Processed prompts: 100%|██████████| 7/7 [00:12<00:00,  1.74s/it, est. speed input: 565.68 toks/s, output: 197.57 toks/s]


merging all enrichment dictionaries...


100%|██████████| 7/7 [00:00<00:00, 33250.43it/s]

Time taken: 13.81446099281311 seconds (0.23024101654688517 minutes)


In [29]:
updated_dict = taxo.toDict(cur_node=taxo.root)
with open(f'datasets/{args.dataset}/enriched.json', 'w') as fp:
    json.dump(updated_dict, fp, indent=4)

In [30]:
with open("preprocessing/AutoPhrase/data/EN/wiki_quality_orig.txt", "r", encoding='utf-8') as f:
    all_phrases = [w.strip() for w in f.readlines() if " " in w.strip()]
    for a in list(taxo.label2id.keys()) + all_common_phrases:
        all_phrases.append(a.strip().replace("_", " "))

with open("preprocessing/AutoPhrase/data/EN/wiki_quality.txt", "w", encoding='utf-8') as f:
    for w_id, w in enumerate(all_phrases):
        if w_id == (len(all_phrases) - 1):
            f.write(f"{w}")
        else:
            f.write(f"{w}\n")

In [31]:
if args.override:
    # pre-process
    os.chdir("./preprocessing")
    subprocess.check_call(['./auto_phrase.sh', args.dataset, args.internal])
    subprocess.check_call(['./auto_phrase.sh', args.dataset, args.external])
    os.chdir("../")
else:
    print("already pre-processed!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


===Corpus Pre-processing===


100%|██████████| 34/34 [00:01<00:00, 22.55it/s]


===Compilation===
===Tokenization===



real	0m1.546s
user	0m12.814s
sys	0m0.669s


...

Detected Language: EN
===Part-Of-Speech Tagging===
Current step: Merging...s...
===Phrasal Segmentation===


=== Current Settings ===
Segmentation Model Path = models/NEW/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.700000
	Q(single-word phrases) >= 1.000000
POS guided model loaded.
# of loaded patterns = 24515
# of loaded truth patterns = 35795
POS transition matrix loaded
Phrasal segmentation finished.
   # of total highlighted quality phrases = 104679
   # of total processed sentences = 189417
   avg highlights per sentence = 0.552638

real	0m2.421s
user	0m2.359s
sys	0m0.056s


===Generating Output===
===Segmented Corpus Post-processing===


8150it [00:00, 23559.30it/s]


Phrase segmented corpus written to ../datasets/gen_kgc/phrase_gen_kgc_external.txt


In [32]:
# taxo.static_emb = {}
# taxo.token_lens = {}
# taxo.collection = []
# taxo.external_collection = []

In [33]:
collection, external_collection = taxo.createCollections(args)

Constructing Collections...


100%|██████████| 8184/8184 [00:05<00:00, 1368.69it/s]


Computing Static Embeddings...
Loading in static embeddings and updating...


100%|██████████| 8184/8184 [00:01<00:00, 8148.53it/s] 


**External & Internal Term Enrichment**

In [34]:
print("encoding sentences...")
external_sentences = list(set([sentence for paper in external_collection for sentence in paper.sent_tokenize]))
internal_sentences = list(set([sentence for paper in collection for sentence in paper.sent_tokenize]))
taxo.updateVocab(external_sentences + internal_sentences, 'sentences')
# sent2emb = {sent:idx for idx, sent in enumerate(external_sentences)}
# external_sent_emb = {idx:emb for idx, emb in  enumerate(sentence_model.encode(external_sentences))}

print("encoding phrases...")
external_phrases = list(set([phrase for paper in external_collection for sentence in paper.phrase_tokenize for phrase in sentence]))
internal_phrases = list(set([phrase for paper in collection for sentence in paper.phrase_tokenize for phrase in sentence]))
taxo.updateVocab(external_phrases + internal_phrases, 'phrases')
# phrase2emb = {phrase:idx for idx, phrase in enumerate(external_phrases)}
# external_phrase_emb = {idx:emb for idx, emb in  enumerate(sentence_model.encode(external_phrases))}

taxo.graph.external['phrases'] = external_phrases
taxo.graph.external['sentences'] = external_sentences
taxo.graph.internal['phrases'] = internal_phrases
taxo.graph.internal['sentences'] = internal_sentences

encoding sentences...
encoding phrases...


In [35]:
phrase_pool = external_phrases # list(taxo.vocab['phrases'].keys())
pool_emb = np.array([taxo.vocab['phrases'][w] for w in phrase_pool])

In [36]:
node_external_phrase_ranks, gt, preds = expandDiscriminative(taxo, phrase_pool, pool_emb, internal=False)
f1_scores(gt, preds)

100%|██████████| 7/7 [00:04<00:00,  1.51it/s]

0.8069879466938291
F1-Macro Score: 0.6421009098428454
F1-Micro Score: 0.7984189723320158


In [37]:
term_to_idx, td_matrix, co_matrix = constructTermDocMatrix(taxo, collection + external_collection)
co_avg = np.true_divide(co_matrix.sum(),(co_matrix!=0).sum())
phrase_pool = internal_phrases # list(taxo.vocab['phrases'].keys())
pool_emb = np.array([taxo.vocab['phrases'][w] for w in phrase_pool])

100%|██████████| 8184/8184 [00:21<00:00, 373.31it/s]


In [38]:
bm_score = computeBM25Cog(co_matrix, co_avg)

In [39]:
node_internal_phrase_ranks, gt, preds = expandInternal(taxo, phrase_pool, pool_emb, term_to_idx, bm_score)
f1_scores(gt, preds)

100%|██████████| 7/7 [00:32<00:00,  4.69s/it]

0.801139546727782
F1-Macro Score: 0.7246886446886448
F1-Micro Score: 0.7956204379562044


**Internal Sentence Enrichment**

In [40]:
sentence_pool = []
sentence_gold = []
sentence_phrase_pool = []
for paper in collection:
    for s_sent, p_sent in zip(paper.sent_tokenize, paper.phrase_tokenize):
        if s_sent not in sentence_pool:
            sentence_pool.append(s_sent)
            sentence_gold.append(paper.gold)
            sentence_phrase_pool.append(p_sent)

phrase_pool_emb = [np.stack([taxo.vocab['phrases'][w] for w in s], axis=0) for s in sentence_phrase_pool] # S x P x 768
sentence_pool_emb = np.array([taxo.vocab['sentences'][s] for s in sentence_pool])

In [51]:
taxo.root.internal['sentences'] = sentence_pool
taxo.root.internal['sent_ids'] = np.arange(len(sentence_pool))
queue = deque([taxo.root])

while queue:
    curr_node = queue.popleft()
    # for each child, compute phrase emb and sib emb
    for child in curr_node.children:
        child.internal['sentences'] = []
        child.internal['sent_ids'] = []

        child.emb['phrase'] = np.stack([taxo.vocab['phrases'][w] 
                                        for w in child.getAllTerms(granularity='phrases', children=False)], axis=0)
        child.emb['sentence'] = np.stack([taxo.vocab['sentences'][w] 
                                        for w in child.getAllTerms(granularity='sentences', children=False)], axis=0)
    
    candidate_phrases = sentence_phrase_pool # [sentence_phrase_pool[i] for i in curr_node.internal['sent_ids']]
    candidate_phrase_embs = phrase_pool_emb #[phrase_pool_emb[i] for i in curr_node.internal['sent_ids']]
    candidate_sent_embs = sentence_pool_emb #sentence_pool_emb[curr_node.internal['sent_ids']]

    sent_ranks = {sent_id:[] for sent_id in np.arange(len(sentence_pool_emb))} # for each candidate: list of ranks across all child nodes

    for focus_node in tqdm(curr_node.children):
        sibs = [n for n in curr_node.children if n != focus_node]

        focus_phrases = focus_node.getAllTerms(granularity='phrases', children=False)
        sibling_phrases = [sib.getAllTerms(granularity='phrases', children=False) for sib in sibs]
        
        # compute target phrase/sentence semantic similarity
        focus_phrase_sim = np.stack([cosine_similarity_embeddings(p_embs, focus_node.emb['phrase']).max(axis=0) 
                                     for p_embs in candidate_phrase_embs], axis=0) # S x [P x N] -> S x N
        avg_focus_phrase_sim = average_with_harmonic_series(focus_phrase_sim, axis=1)  # S x 1

        focus_sent_sim = cosine_similarity_embeddings(candidate_sent_embs, focus_node.emb['sentence']) # S x N
        avg_focus_sent_sim = average_with_harmonic_series(focus_sent_sim, axis=1)  # S x 1

        # compute co_occurrence with focus node
        target_co_occurrence = np.array([average_with_harmonic_series(getBM25(sent, focus_phrases, term_to_idx, bm_score).mean(axis=0)) 
                                         for sent in candidate_phrases]) # S x 1
        
        # compute sibling sentence semantic dissimilarity
        sib_phrase_sims = [np.stack([cosine_similarity_embeddings(p_embs, sib.emb['phrase']).max(axis=0)
                                      for p_embs in candidate_phrase_embs], axis=0)
                                      for sib in sibs] # siblings x sentences x P x N -> sib x sentences x N
        sib_sent_sims = [cosine_similarity_embeddings(candidate_sent_embs, sib.emb['sentence']) for sib in sibs] # siblings x sentences x sib_sents
        
        if len(sibs):
            avg_sib_phrase_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_phrase_sims], axis=-1).max(axis=1) # sentences x 1
            avg_sib_sent_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_sent_sims], axis=-1).max(axis=1) # sentences x 1
            # compute sibling co-occurrence
            sib_co_occurrence = np.array([max([average_with_harmonic_series(getBM25(sent_phrases, sib_terms, term_to_idx, bm_score).mean(axis=0)) 
                                               for sib_terms in sibling_phrases]) for sent_phrases in candidate_phrases]) # S x 1
        else:
            avg_sib_phrase_sim = np.zeros_like(avg_focus_phrase_sim)
            avg_sib_sent_sim = np.zeros_like(avg_focus_sent_sim)
            sib_co_occurrence = np.zeros_like(target_co_occurrence)
            
        # compute semantic rank
        target_sim_phrase_rank = {idx:rank for rank, idx in enumerate((avg_focus_phrase_sim-avg_sib_phrase_sim).argsort()[::-1])}
        target_sim_sent_rank = {idx:rank for rank, idx in enumerate((avg_focus_sent_sim-avg_sib_sent_sim).argsort()[::-1])}
        
        # compute co-occurrence rank
        target_co_rank = {idx:rank for rank, idx in enumerate((target_co_occurrence-sib_co_occurrence).argsort()[::-1])}

        joint_rank = compute_joint_ranking([target_sim_phrase_rank, target_sim_sent_rank, target_co_rank]) # arr idx: rank

        for idx in np.arange(len(sentence_pool)):
            sent_ranks[idx].append(joint_rank[idx])

    # filter sentences based on rank
    for node_id, focus_node in enumerate(curr_node.children):
        sorted_ranks = sorted([s_id 
                               for s_id in np.arange(len(sentence_pool)) 
                               if (sent_ranks[s_id][node_id] <= min(sent_ranks[s_id])) 
                               and (len(sentence_phrase_pool[s_id]) > 5) 
                               and ((sent_ranks[s_id][node_id]) < len(sent_ranks)//2)], 
                               key=lambda x: sent_ranks[x][node_id])

        focus_node.internal['sentences'] = [sentence_pool[s_id] for s_id in sorted_ranks]
        focus_node.internal['sent_ids'] = sorted_ranks
        queue.append(focus_node)

100%|██████████| 5/5 [02:20<00:00, 28.06s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [56]:
node_id = 4
taxo.root.children[0].children[node_id], taxo.root.children[0].children[node_id].internal['sentences']

(knowledge_graph_completion,
 ['from discrimination to generation : knowl__edge graph completion with generative_transformer',
  'composition_based multi_ relational graph convolutional networks',
  '; paper_content : from discrimination to generation : knowledge_graph completion with generative_transformer xin xie zhejiang_university & azft joint lab for knowledge engine hangzhou , china xx2020 @ zju.edu.cnningyu zhang zhejiang_university & azft joint lab for knowledge engine hangzhou , china zhangningyu @ zju.edu.cnzhoubo li , shumin_deng zhejiang_university & azft joint lab for knowledge engine hangzhou , china { zhoubo.li,231sm } @ zju.edu.cn hui chen alibaba_group hangzhou , china weidu.ch @ alibaba_inc.comfeiyu xiong , mosha chen alibaba_group hangzhou , china { feiyu.xfy , chenmosha.cms } @ alibaba_ inc.comhuajun chen zhejiang_university & azft joint lab for knowledge engine hangzhou , china huajunsir @ zju.edu.cn abstract knowledge_graph completion aims to address the problem o

**Paper Classification**

In [ ]:
for paper in collection:
    for child in curr_node.children:
        child.emb['phrase'] = np.stack([taxo.vocab['phrases'][w] 
                                        for w in child.getAllTerms(granularity='phrases', children=False)], axis=0)
        child.emb['sentence'] = np.stack([taxo.vocab['sentences'][w] 
                                        for w in child.getAllTerms(granularity='sentences', children=False)], axis=0)
    # rank sentences based on semantic sim
    paper.rank

    # compute paper embedding based discriminative sentence weights

    # for each node, assign papers based on similarity gap → take top 50% of assigned papers as pseudo-labels

    # fine-tune classifier

In [38]:
def rankSentences(taxo, sents, sent_phrases, phrase_embs, sent_embs, term_to_idx, bm_score, thresh=3, min_freq=3, percentile=0.999, classify=True):
    
    node_text_ranks = []

    for node_id in tqdm(np.arange(0, len(taxo.label2id))):
        # gather node and its siblings
        focus_node = taxo.root.findChild(str(node_id))
        sibling_nodes = taxo.get_sib(focus_node.node_id, granularity='emb')

        # get phrases of node and its siblings
        focus_phrase = focus_node.getAllTerms(granularity='phrases', children=False)
        focus_phrase_embs = np.stack([taxo.vocab['phrases'][w] for w in focus_phrase], axis=0) # P x dim
        sibling_phrase = [sib.getAllTerms(granularity='phrases', children=False) for sib in sibling_nodes] # list of list of sibling node terms
        sib_phrase_embs = [np.stack([taxo.vocab['phrases'][p] for p in t], axis=0) for t in sibling_phrase] # [(T x dim), ... ()] 
        # get sentences of node and its siblings
        focus_sent = focus_node.getAllTerms(granularity='sentences', children=False)
        focus_sent_embs = np.stack([taxo.vocab['sentences'][s] for s in focus_sent], axis=0) # sentences x dim
        sibling_sent = [sib.getAllTerms(granularity='sentences', children=False) for sib in sibling_nodes] # list [(list of sibling node i terms), ...]
        sib_sent_embs = [np.stack([taxo.vocab['sentences'][s] for s in t], axis=0) for t in sibling_sent] # list [(# of sib sents x dim)]

        # compute target phrase/sentence semantic similarity
        focus_phrase_sim = np.stack([cosine_similarity_embeddings(s, focus_phrase_embs).max(axis=0) for s in phrase_embs], axis=0) # S x [P x N] -> S x N
        avg_focus_phrase_sim = average_with_harmonic_series(focus_phrase_sim, axis=1)  # S x 1

        focus_sent_sim = cosine_similarity_embeddings(sent_embs, focus_sent_embs) # S x N
        avg_focus_sent_sim = average_with_harmonic_series(focus_sent_sim, axis=1)  # S x 1

        # compute sibling sentence semantic dissimilarity
        sib_phrase_sims = [np.stack([cosine_similarity_embeddings(sent_phrase_emb, s_emb).max(axis=0) for sent_phrase_emb in phrase_embs], axis=0) for s_emb in sib_phrase_embs] # siblings x sentences x P x N -> sib x sentences x N
        sib_sent_sims = [cosine_similarity_embeddings(sent_embs, s_emb) for s_emb in sib_sent_embs] # siblings x sentences x sib_sents
        if len(sibling_nodes):
            avg_sib_phrase_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_phrase_sims], axis=-1).max(axis=1) # sentences x 1
            avg_sib_sent_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_sent_sims], axis=-1).max(axis=1) # sentences x 1
        else:
            avg_sib_phrase_sim = np.zeros_like(avg_focus_phrase_sim)
            avg_sib_sent_sim = np.zeros_like(avg_focus_sent_sim)

        # compute semantic rank
        target_sim_phrase_rank = {idx:rank for rank, idx in enumerate((avg_focus_phrase_sim-avg_sib_phrase_sim).argsort()[::-1])}
        target_sim_sent_rank = {idx:rank for rank, idx in enumerate((avg_focus_sent_sim-avg_sib_sent_sim).argsort()[::-1])}

        # compute target co-occurrence
        target_co_occurrence = np.array([average_with_harmonic_series(getBM25(sent, focus_phrase, term_to_idx, bm_score).mean(axis=0)) for sent in sent_phrases]) # S x 1
        
        # compute sibling co-occurrence
        if len(sibling_nodes):
            sib_co_occurrence = np.array([max([average_with_harmonic_series(getBM25(sent, sib_terms, term_to_idx, bm_score).mean(axis=0)) for sib_terms in sibling_phrase]) for sent in sent_phrases]) # S x 1
        else:
            sib_co_occurrence = np.zeros_like(target_co_occurrence)
        
        # compute co-occurrence rank
        target_co_rank = {idx:rank for rank, idx in enumerate((target_co_occurrence-sib_co_occurrence).argsort()[::-1])}

        joint_rank = compute_joint_ranking([target_sim_phrase_rank, target_sim_sent_rank, target_co_rank]) # idx: rank
        sorted_ranks = sorted(joint_rank.items(), key=lambda x: x[1])

        final_ranks = {}
        for idx, rank in sorted_ranks:
            # if rank > (1-percentile)*len(sents):
            #     break
            if (len(sent_phrases[idx]) > 5): # and (avg_focus_phrase_sim[idx] > avg_sib_phrase_sim[idx]) and (avg_focus_sent_sim[idx] > avg_sib_sent_sim[idx]) and (target_co_occurrence[idx] > sib_co_occurrence[idx]):
                final_ranks[rank] = (sents[idx], avg_focus_phrase_sim[idx], avg_focus_sent_sim[idx], target_co_occurrence[idx])
                # focus_node.internal['sentences'].append(text[idx])
        
        node_text_ranks.append(final_ranks)

    return node_text_ranks

In [29]:
node_id = 4
taxo.root.findChild(str(node_id)), taxo.root.findChild(str(node_id)).external['phrases']
# taxo.root.findChild(str(node_id)).getAllTerms(granularity='phrases', children=False)

(event_extraction,
 ['event_level',
  'event_',
  'event_specific',
  'event_type',
  'event_role',
  'event_generation',
  'event_types',
  'event_extraction',
  'event_arguments',
  'multi_event',
  'event_detection',
  'event_argument_extraction',
  'event_record',
  'event_related',
  'event_masked',
  'event_records',
  'event_relevant',
  'event_correlation'])

In [ ]:
queue = deque([taxo.root])
taxo.root.papers = {paper.id:paper for paper in collection}

while queue:
    focus_node = queue.popleft()
    sibling_nodes = taxo.get_sib(focus_node.node_id, granularity='nodes')

    # get phrases of node and its siblings
    focus_phrase = focus_node.getAllTerms(granularity='phrases', children=False)
    focus_phrase_embs = np.stack([taxo.vocab['phrases'][w] for w in focus_phrase], axis=0) # P x dim
    sibling_phrase = [sib.getAllTerms(granularity='phrases', children=False) for sib in sibling_nodes] # list of list of sibling node terms
    sib_phrase_embs = [np.stack([taxo.vocab['phrases'][p] for p in t], axis=0) for t in sibling_phrase] # [(T x dim), ... ()]
    
    # get sentences of node and its siblings
    focus_sent = focus_node.getAllTerms(granularity='sentences', children=False)
    focus_sent_embs = np.stack([taxo.vocab['sentences'][s] for s in focus_sent], axis=0) # sentences x dim
    sibling_sent = [sib.getAllTerms(granularity='sentences', children=False) for sib in sibling_nodes] # list [(list of sibling node i terms), ...]
    sib_sent_embs = [np.stack([taxo.vocab['sentences'][s] for s in t], axis=0) for t in sibling_sent] # list [(# of sib sents x dim)]


In [ ]:
for paper in tqdm(collection):
    # for each sentence in paper
    ## weigh each phrase in sentence based on how discriminative it is
    for sent in paper.phrase_tokenize:
        
        phrase_sim = cosine_similarity_embeddings()

In [21]:
for p in zip(gt, preds):
    print(f'preds: {p[1]}; gt: {p[0]}')

preds: {'0', '1'}; gt: ['0', '1', '2']
preds: {'2', '0', '1'}; gt: ['0', '1', '2']
preds: {'5', '2', '1', '0', '4', '3'}; gt: ['0', '1', '2', '3', '4']
preds: {'3', '0', '6', '1'}; gt: ['0', '1', '2', '3']
preds: {'2', '0', '1'}; gt: ['0', '1', '2']
preds: {'3', '0', '1'}; gt: ['0', '1', '2', '3', '4']
preds: {'2', '0', '1'}; gt: ['0', '1', '2']
preds: {'2', '0', '4', '1'}; gt: ['0', '1', '2', '3', '4']
preds: {'0', '1'}; gt: ['0', '1', '3']
preds: {'0', '1'}; gt: ['0', '1', '3']
preds: {'3', '2', '0', '1'}; gt: ['0', '1', '3']
preds: {'0', '1'}; gt: ['0', '1', '3']
preds: {'1', '0', '4', '3', '6'}; gt: ['0', '1', '3', '4']
preds: {'0', '1'}; gt: ['0', '1', '3', '4']
preds: {'2', '0', '1'}; gt: ['0', '1', '3']
preds: {'3', '5', '0', '1'}; gt: ['0', '1', '3']
preds: {'5', '0', '1'}; gt: ['0', '1', '3']
preds: {'0', '4', '1'}; gt: ['0', '1', '4']
preds: {'0', '4', '1'}; gt: ['0', '1', '4']
preds: {'5', '0', '4', '1'}; gt: ['0', '1', '4']
preds: {'0', '4', '1'}; gt: ['0', '1', '4']
preds:

In [202]:
# sentence_pool = internal_sentences
# sentence_pool_emb = np.array([taxo.vocab['sentences'][s] for s in sentence_pool])

In [39]:
# node_external_sent_ranks = expandDiscriminative(taxo, sentence_pool, sentence_pool_emb, granularity='sentences', classify=False, internal=-1)
node_internal_sent_ranks = rankSentences(taxo, sentence_pool, sentence_phrase_pool, phrase_pool_emb, sentence_pool_emb, term_to_idx, bm_score)

100%|██████████| 7/7 [02:28<00:00, 21.24s/it]


In [96]:
taxo.label2id

{'generative_knowledge_graph_construction': '0',
 'generation_tasks': '1',
 'named_entity_recognition': '2',
 'relation_extraction': '3',
 'event_extraction': '4',
 'entity_linking': '5',
 'knowledge_graph_completion': '6'}

In [58]:
for k,v in node_internal_sent_ranks[4].items():
    print(k, v)
    if k > 100:
        break

1 ('[ 8 ] propose a dynamic multi_pooling convolutional neural_network ( dm_ cnn ) , which uses a dynamic multi_pooling layer according to event triggers and arguments for event_extraction', 0.913306785813715, 0.7770894100358857, 0.0031051727799206123)
3 ('the argument part is specic to event_type ei', 0.914617693454433, 0.7869675802903486, 0.002727315162633439)
5 ('the second group contains generation_based event_extraction methods', 0.9080140282650968, 0.8621444665243786, 0.0024849446481576497)
6 ('paper_title : text2event : controllable sequence_to_structure generation for end_to_end event_extraction ; paper_abstract : event_extraction is challenging due to the complex structure of event_records and the semantic gap between text and event', 0.9132553905853479, 0.8347001638762626, 0.0025721657225129344)
7 (', text2event : controllable sequence_to_structure generation for end_to_end event_extraction , in proc', 0.9081088541185993, 0.814699993799282, 0.0028579141235849856)
8 ('2 t ext2

In [41]:
taxo.resetTaxo()

In [42]:
queue = deque([taxo.root])

In [43]:
while queue:
    curr_node = queue.popleft()
    print(f"PROCESSING {curr_node.label}; remaining in queue: {queue}")

    # update vocab with label and description
    taxo.updateVocab([curr_node.label] + curr_node.description.split(), 'phrases')
    taxo.updateVocab(curr_node.description, 'sentences')
    taxo.updateVocab(curr_node.label + " : " + curr_node.description, 'sentences')

    # get common-sense class embedding
    print('before:', curr_node.common_sense['phrases'])
    top_common_phrases, common_phrase_node_emb = computeClassEmb(curr_node, taxo, granularity='phrases', out_phrases=True)
    top_common_sentences, common_sent_node_emb = computeClassEmb(curr_node, taxo, granularity='sentences', out_phrases=True)

    print('after:', top_common_phrases)
    # curr_node.common_sense['phrases'] = top_common_phrases
    # curr_node.common_sense['sentences'] = top_common_sentences

    for child in curr_node.children:
        queue.append(child)

PROCESSING generative_knowledge_graph_construction; remaining in queue: deque([])
before: ['graph_construction', 'knowledge_graph_building', 'generative_knowledge_graph', 'graph_generation', 'graph_building', 'graph_construction_tasks', 'generative_knowledge', 'graph_construction_techniques', 'knowledge_graph_construction', 'graph_building_tasks', 'graph_construction_methods', 'generative_knowledge_graph_construction_techniques', 'graph_construction_approaches', 'knowledge_graph_generation']
after: ['generative_knowledge_graph_construction', 'generative_knowledge_graph_construction_techniques', 'generative_knowledge_graph', 'generative_knowledge', 'knowledge_graph_generation', 'knowledge_graph_construction', 'knowledge_graph_building', 'graph_generation', 'graph_construction_tasks', 'graph_construction_approaches', 'graph_construction', 'graph_construction_techniques', 'graph_construction_methods', 'graph_building', 'graph_building_tasks']
PROCESSING generation tasks; remaining in queu

In [66]:
paper_map = {n.node_id:[] for n in taxo.root.children[0].children}

for p in collection + external_collection:
    for n in taxo.root.children[0].children:
        if any([term in p.vocabulary for term in n.common_sense['phrases']]):
            paper_map[n.node_id].append(p)

In [67]:
for n_id, paper_list in paper_map.items():
    print(n_id, len(paper_list))

2 235
3 56
4 27
5 115
6 119


In [63]:
paper_map["4"]

[id: 2; title: structured_prediction as translation between augmented natural languages; abstract: we propose a new framework , translation between augmented natural languages ( tanl ) , to solve many structured_prediction language tasks including joint_entity_and_relation_extraction , nested named_entity recognition , relation_classification , semantic_role_labeling , event_extraction , coreference_resolution , and dialogue_state_tracking . instead of tackling the problem by training task_specific discriminative classifiers , we frame it as a translation task between augmented natural languages , from which the task_relevant information can be easily extracted . our approach can match or outperform task_specific models on all tasks , and in particular , achieves new state_of_the_art results on joint_entity_and_relation_extraction ( conll04 , ade , nyt , and ace2005 datasets ) , relation_classification ( fewrel and tacred ) , and semantic_role_labeling ( conll_2005 and conll_2012 ) . w

In [187]:
from main import rankPhrases

In [206]:
ranked_external, class_emb = rankPhrases(external_sentences, taxo.root, taxo, use_class_emb=True, granularity='sentences', out_phrases=True)

In [33]:
queue = deque([taxo.root])

In [34]:
while queue:
    curr_node = queue.popleft()
    parent_node = curr_node.parents[0]

    print(f"PROCESSING {curr_node.label}; remaining in queue: {queue}")

    # EXTERNAL: get partitioned set of external phrases and sentences
    curr_node.external['phrases'] = external_phrases # parent_node.external['phrases']
    curr_node.external['sentences'] = external_sentences # parent_node.external['sentences']
  
    # get top external phrases & external sentences
    top_phrases, external_phrase_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='phrases', out_phrases=True)
    top_sentences, external_sentence_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='sentences', out_phrases=True)

    top_external_phrases = [phrase for phrase in top_phrases if phrase in curr_node.external['phrases']]
    top_external_sentences = [sent for sent in top_sentences if sent in curr_node.external['sentences']]
    curr_node.external['phrases'] = top_external_phrases
    curr_node.external['sentences'] = top_external_sentences

    # INTERNAL: get partitioned set of internal phrases and sentences
    curr_node.internal['phrases'] = internal_phrases # parent_node.internal['phrases']
    curr_node.internal['sentences'] = internal_sentences # parent_node.internal['sentences']

    # get top internal phrases & sentences
    top_phrases, internal_phrase_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='phrases', out_phrases=True)
    top_sentences, internal_sentence_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='sentences', out_phrases=True)

    top_internal_phrases = [phrase for phrase in top_phrases if phrase in curr_node.internal['phrases']]
    top_internal_sentences = [sent for sent in top_sentences if sent in curr_node.internal['sentences']]
    curr_node.internal['phrases'] = top_internal_phrases
    curr_node.internal['sentences'] = top_internal_sentences

    for child in curr_node.children:
        queue.append(child)

PROCESSING generative_knowledge_graph_construction; remaining in queue: deque([])
PROCESSING generation tasks; remaining in queue: deque([])
PROCESSING named_entity_recognition; remaining in queue: deque([relation_extraction, event_extraction, entity_linking, knowledge_graph_completion])
PROCESSING relation_extraction; remaining in queue: deque([event_extraction, entity_linking, knowledge_graph_completion])
PROCESSING event_extraction; remaining in queue: deque([entity_linking, knowledge_graph_completion])
PROCESSING entity_linking; remaining in queue: deque([knowledge_graph_completion])
PROCESSING knowledge_graph_completion; remaining in queue: deque([])


In [40]:
taxo.root.children[0].children[0].external

{'phrases': ['entity_recognition',
  'bag_of_entity',
  'bag_of_entities',
  'named',
  'anti_nose',
  'tokens',
  'artificial_neural_networks',
  'facial_expression',
  '//github.com/wjn1996/mathematical_knowledge_entity_recognition',
  'artificial_intelligence',
  'andrecognize',
  'token',
  'murder',
  'tokenization',
  'negated',
  'mention_pronoun',
  'word_vector',
  'weaponry',
  'inflated',
  'optical_character_recognition',
  'token_based',
  "'phoneme",
  'word_character',
  'natural_language',
  'intent_classifier',
  'entity_classification',
  'entity_detection',
  'mention',
  'name',
  'non_named',
  'word_by_word',
  'weapon',
  'named_entity_extraction',
  'noun',
  'sign_language',
  'applications_of_artificial_intelligence',
  'noun_phrase',
  'intelligent_agent',
  'pronoun',
  'character_vocabulary',
  'word_entity',
  'support_vector_machine',
  'utterances',
  'automatic_speech_recognition',
  'naming',
  'chinese_language',
  'uttered',
  'controlled_vocabulary'

In [265]:
ranked_papers = taxo.rankPapers([curr_node.emb['sentences'] for curr_node in taxo.root.children], 2, internal=True, phrase=False, top_k=20)

In [266]:
ranked_papers

[(66, 0),
 (65, 1),
 (71, 2),
 (69, 3),
 (51, 4),
 (47, 5),
 (55, 6),
 (50, 7),
 (48, 8),
 (70, 9),
 (24, 10),
 (54, 11),
 (4, 12),
 (62, 13),
 (25, 14),
 (11, 15),
 (22, 16),
 (53, 17),
 (67, 18),
 (59, 19)]

In [286]:
paper_embs = {idx: paper_emb for idx, paper_emb in enumerate(sentence_model.encode([paper.title for paper in collection]))}

In [306]:
queue = deque([taxo.root])

In [307]:
class_labels = {}
class_mappings = {}
paper_scores = {}

while queue:
    target_node = queue.popleft()
    print(f"PROCESSING {target_node.label}; remaining in queue: {queue}")

    class_ids = [child.node_id for child in target_node.children]
    print(class_ids)

    if len(target_node.children) <= 1:
        continue
    
    ranked_papers = []
    for idx in np.arange(len(class_ids)):
        ranked_papers.append(taxo.rankPapers([curr_node.emb['sentences'] for curr_node in target_node.children], idx, internal=True, phrase=False, top_k=20))
    
    for p in ranked_papers:
        print("next!")
        for c in p:
            print(collection[c[0]].title)

    ranked_paper_embs = [average_with_harmonic_series([paper_embs[p[0]] for p in c]) for c in ranked_papers]


    node_labels, node_paper_scores, node_mapping = taxo.mapPapers(np.array(list(paper_embs.values())), target_node.children, ranked_paper_embs)

    class_labels[target_node.node_id] = node_labels
    paper_scores[target_node.node_id] = node_paper_scores
    class_mappings[target_node.node_id] = node_mapping


    # full_text_sim = cosine_similarity_embeddings(list(paper_embs.values()), ranked_paper_embs)
    # full_text_preds = full_text_sim.argmax(axis=1)
    # full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

    # for idx, paper_id in enumerate(target_node.papers):
    #     target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

    print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

    for child in target_node.children:
        queue.append(child)

PROCESSING using_llms_on_graphs; remaining in queue: deque([])
['1', '2', '3']
next!
node feature_extraction by self_supervised multi_scale neighborhood_prediction
chemformer : a pre_trained transformer for computational_chemistry
catalyst property_prediction with catberta : unveiling feature exploration strategies through large_language_models
efficient and effective training of language and graph_neural_network models
regression transformer enables concurrent sequence regression and generation for molecular language_modelling
graph_toolformer : to empower llms with graph_reasoning ability via prompt augmented by chatgpt
llm4dyg : can large_language_models solve spatial_temporal problems on dynamic_graphs ?
label_free node_classification on graphs with large_language_models ( llms )
can language_models solve_graph_problems in natural_language ?
relm : leveraging language_models for enhanced chemical_reaction prediction
text2mol : cross_modal molecule retrieval with natural_language qu

In [ ]:
collection[0].rankSentences()

In [308]:
class_labels

{'0': [[1],
  [0, 1],
  [1, 0],
  [0],
  [1],
  [0, 1],
  [1],
  [0, 1],
  [0],
  [],
  [1],
  [1],
  [1],
  [1],
  [0],
  [0, 1],
  [1],
  [1],
  [1],
  [1],
  [1],
  [1],
  [0, 1],
  [1],
  [0],
  [],
  [1],
  [],
  [0],
  [1],
  [1],
  [0, 1],
  [0],
  [0, 1],
  [1],
  [0, 1],
  [1, 0],
  [1],
  [1],
  [1],
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1],
  [1],
  [0],
  [],
  [0],
  [],
  [],
  [],
  [1],
  [0],
  [],
  [],
  [1],
  [0],
  [],
  [0],
  [0],
  [0],
  [],
  [1],
  [],
  [2],
  [2],
  [],
  [0],
  [],
  [],
  [2]],
 '2': [[1, 2],
  [2],
  [2],
  [2],
  [2],
  [1, 2],
  [1, 2],
  [2],
  [2],
  [],
  [1, 2],
  [2, 1],
  [2, 1],
  [1, 2],
  [2],
  [1, 2],
  [2, 1],
  [2, 1],
  [0, 1],
  [1],
  [1, 0],
  [1],
  [0, 1],
  [1],
  [0],
  [],
  [0],
  [0],
  [],
  [],
  [],
  [2, 1],
  [0],
  [2, 1],
  [1],
  [1, 2],
  [2],
  [1],
  [1],
  [1],
  [0],
  [2],
  [0, 1],
  [1],
  [0, 1],
  [1],
  [0],
  [2],
  [2, 0],
  [],
  [],
  [0],
  [1, 0],
  [2],
  [1, 0],
  [],
  [0],
 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0])

In [65]:
from utils import rank_by_class_discriminative_significance, filter_by_class_discriminative_significance

In [97]:
phrase_reprs = np.concatenate([taxo.static_emb[w].reshape((-1, 768)) for w in internal_phrases], axis=0)
ranks = rank_by_class_discriminative_significance(phrase_reprs, [average_with_harmonic_series([taxo.static_emb[w] for w in c.external['phrases']]) for c in focus_node.parents[0].children], focus_node.parents[0].children.index(focus_node))
ranked_tok = {internal_phrases[idx]:rank for idx, rank in ranks.items()}

ZeroDivisionError: Weights sum to zero, can't be normalized

In [98]:
ranked_tok

{'learning': 0,
 'reasoning': 1,
 'classification': 2,
 'retrieval': 3,
 'inference': 4,
 'smoothing': 5,
 'graph': 6,
 'model': 7,
 'knowledge': 8,
 'representations': 9,
 'policy': 10,
 'representation': 11,
 'based': 12,
 'understanding': 13,
 'cross': 14,
 'analysis': 15,
 'models': 16,
 'representation_learning': 17,
 'taxonomy': 18,
 'routing': 19,
 'function': 20,
 'generation': 21,
 'grouping': 22,
 'encoding': 23,
 'network': 24,
 'trajectory': 25,
 'dialogue': 26,
 'dictionary': 27,
 'using': 28,
 'discourse': 29,
 'algorithm': 30,
 'derivation': 31,
 'search': 32,
 'with': 33,
 'estimates': 34,
 'modeling': 35,
 'attention': 36,
 'prompting': 37,
 'targets': 38,
 'molecular': 39,
 'rating': 40,
 'programming': 41,
 'scene': 42,
 'transmission': 43,
 'feature': 44,
 'information': 45,
 'for': 46,
 'task': 47,
 'semantic': 48,
 'collaboration': 49,
 'hierarchy': 50,
 'conditioned': 51,
 'reward': 52,
 'action': 53,
 'correction': 54,
 'searches': 55,
 'structure': 56,
 'orderi

In [96]:
focus_node = taxo.root.findChild("18")
focus_node.label

'data_augmentation'

In [36]:
focus_node.external['phrases']

['answer_set',
 'answer_prediction',
 'answer_generation',
 'invoices',
 'answering_systems',
 'image_question',
 'answer_candidates',
 'answer_answer',
 'disengagement',
 'conversation_generation',
 'answer_contained',
 'chatting',
 'non_mediated',
 'present_participle',
 'past_participle',
 'soliciting',
 'dialogue_generation',
 'intolerant',
 'donations',
 'candidate_answers',
 'hate_speech',
 'self_distillation',
 'dialogue_interaction',
 'teacher_forcing',
 'know_why',
 'exclamation',
 'sub_questions',
 'canencourage',
 'question_comment',
 'complex_questions',
 'never_before_seen',
 'question2subgraph',
 'redirected',
 'help_seekers',
 'query_answering',
 'unanswerable',
 'frustratingly',
 'quests',
 'hate_keyword',
 'dialogue_management',
 'multi_teacher',
 'aroused',
 'non_directly',
 'few/zero_shot',
 'weakly_annotated',
 'arouse',
 'answering_complex',
 'dining',
 'helpdesks',
 'missing_links',
 'redirection',
 'hop_attention',
 'audiences',
 'delegate',
 'conversational_agen

In [30]:
paper_embs = {idx: paper_emb for idx, paper_emb in enumerate(sentence_model.encode([paper.title for paper in collection]))}

In [31]:
queue = deque([taxo.root])

In [32]:
while queue:
    target_node = queue.popleft()
    print(f"PROCESSING {target_node.label}; remaining in queue: {queue}")

    class_ids = [child.node_id for child in target_node.children]
    print(class_ids)

    phrase_class_embs = [computeClassEmb(curr_node, taxo, granularity='phrases') for curr_node in target_node.children]
    sent_class_embs = [computeClassEmb(curr_node, taxo, granularity='sentences') for curr_node in target_node.children]
    joint_class_embs = [(p+s)/2 for p, s in zip(phrase_class_embs, sent_class_embs)]

    encoded_papers = np.array([paper_embs[p_id] for p_id in target_node.papers])
    phrase_sim = cosine_similarity_embeddings(encoded_papers, phrase_class_embs) # P x C
    sent_sim = cosine_similarity_embeddings(encoded_papers, sent_class_embs)
    joint_sim = cosine_similarity_embeddings(encoded_papers, joint_class_embs)

    phrase_winner = phrase_sim.argmax(axis=1) # P x 1
    sent_winner = sent_sim.argmax(axis=1)
    joint_winner = joint_sim.argmax(axis=1)
    winner_idxs = stats.mode(np.stack((phrase_winner, sent_winner, joint_winner), axis=1), axis=1, keepdims=False)[0]
    labels = [class_ids[winner_idx] for winner_idx in winner_idxs]
    correct = [paper_id in target_node.children[winner_idxs[idx]].gold for idx, paper_id in enumerate(target_node.papers)]
    print("before ranking-based classification: ", sum(correct)/len(correct))

    scores = (phrase_sim[np.arange(len(phrase_sim)), 
                        winner_idxs] + sent_sim[np.arange(len(sent_sim)), 
                                                winner_idxs] + joint_sim[np.arange(len(joint_sim)), 
                                                                        winner_idxs])/3

    class_map = {i:[] for i in class_ids}
    for idx, paper_id in enumerate(target_node.papers):
        class_map[labels[idx]].append((scores[idx], target_node.papers[paper_id]))

    class_map = {i:sorted(class_map[i], key=lambda x: -x[0]) for i in class_ids}
    full_text_class_embs = []
    for c_id in tqdm(class_ids):
        full_text_class_embs.append(average_with_harmonic_series(np.array([paper_embs[p[1].id] for p in class_map[c_id]])))

    full_text_sim = cosine_similarity_embeddings(np.array([paper_embs[p_id] for p_id in target_node.papers]), full_text_class_embs)
    full_text_preds = full_text_sim.argmax(axis=1)
    full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

    for idx, paper_id in enumerate(target_node.papers):
        target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

    print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

    for child in target_node.children:
        queue.append(child)

PROCESSING using_llms_on_graphs; remaining in queue: deque([])
['1', '2', '3']
before ranking-based classification:  0.6111111111111112


100%|██████████| 3/3 [00:00<00:00, 4096.00it/s]

ranking-based classification 0.7916666666666666
PROCESSING pure_graphs; remaining in queue: deque([text_rich_graphs, text_paired_graphs])
['4']


before ranking-based classification:  0.35714285714285715


100%|██████████| 1/1 [00:00<00:00, 2270.87it/s]

...

ranking-based classification 0.0
PROCESSING llm_as_aligner; remaining in queue: deque([llm_as_predictor, llm_as_aligner, direct_answering, heuristic_reasoning, algorithmic_reasoning, graph_as_sequence, graph_empowered_llm, graph_aware_llm_finetuning, optimization, data_augmentation, knowledge_distillation])
['21', '22']
before ranking-based classification:  0.0


100%|██████████| 2/2 [00:00<00:00, 4718.00it/s]


ranking-based classification 0.0
PROCESSING llm_as_predictor; remaining in queue: deque([llm_as_aligner, direct_answering, heuristic_reasoning, algorithmic_reasoning, graph_as_sequence, graph_empowered_llm, graph_aware_llm_finetuning, optimization, data_augmentation, knowledge_distillation, prediction_alignment, latent_space_alignment])
['24', '25']
before ranking-based classification:  0.0


  0%|          | 0/2 [00:00<?, ?it/s]


ZeroDivisionError: Weights sum to zero, can't be normalized

In [52]:
[p[1].id for p in class_map[c_id]]

[8, 2, 7, 3, 59, 49, 47]

In [ ]:
full_text_class_embs.append(average_with_harmonic_series(np.array([encoded_papers[p[1].id] for p in class_map[c_id]])))

In [36]:
target_node = taxo.root#.children[1]

class_ids = [child.node_id for child in target_node.children]
print(class_ids)

phrase_class_embs = [computeClassEmb(curr_node, taxo, granularity='phrases') for curr_node in target_node.children]
sent_class_embs = [computeClassEmb(curr_node, taxo, granularity='sentences') for curr_node in target_node.children]
# joint_class_embs = [computeClassEmb(curr_node, taxo, granularity='mixed') for curr_node in target_node.children]
joint_class_embs = [(p+s)/2 for p, s in zip(phrase_class_embs, sent_class_embs)]

['8', '14', '20']
llm_as_predictor: 57 phrases filtered!
llm_as_encoder: 75 phrases filtered!
llm_as_aligner: 27 phrases filtered!
llm_as_predictor: 46 sentences filtered!
llm_as_encoder: 12 sentences filtered!
llm_as_aligner: 18 sentences filtered!


In [18]:
external_phrase_embs, sim_diff, keep_phrase = compareClasses(external_phrases, taxo, target_node.children[0], 'phrases')
filtered_embs = external_phrase_embs[keep_phrase, :]
filtered_external_phrases = list(compress(external_phrases, keep_phrase))
filtered_diffs = sim_diff[keep_phrase]
print(f'{target_node.label}: {len(external_phrases) - len(filtered_external_phrases)} phrases filtered!')

text_rich_graphs: 6586 phrases filtered!


In [14]:
encoded_papers = sentence_model.encode([collection[p_id].raw_text for p_id in target_node.papers])
phrase_sim = cosine_similarity_embeddings(encoded_papers, phrase_class_embs) # P x C
sent_sim = cosine_similarity_embeddings(encoded_papers, sent_class_embs)
joint_sim = cosine_similarity_embeddings(encoded_papers, joint_class_embs)

phrase_winner = phrase_sim.argmax(axis=1) # P x 1
sent_winner = sent_sim.argmax(axis=1)
joint_winner = joint_sim.argmax(axis=1)
winner_idxs = stats.mode(np.stack((phrase_winner, sent_winner, joint_winner), axis=1), axis=1, keepdims=False)[0]
labels = [class_ids[winner_idx] for winner_idx in winner_idxs]
correct = [paper_id in target_node.children[winner_idxs[idx]].gold for idx, paper_id in enumerate(target_node.papers)]
print("before ranking-based classification: ", sum(correct)/len(correct))

scores = (phrase_sim[np.arange(len(phrase_sim)), 
                    winner_idxs] + sent_sim[np.arange(len(sent_sim)), 
                                            winner_idxs] + joint_sim[np.arange(len(joint_sim)), 
                                                                      winner_idxs])/3

class_map = {i:[] for i in class_ids}
for idx, paper_id in enumerate(target_node.papers):
    class_map[labels[idx]].append((scores[idx], target_node.papers[paper_id]))

class_map = {i:sorted(class_map[i], key=lambda x: -x[0]) for i in class_ids}
full_text_class_embs = []
for c_id in tqdm(class_ids):
    full_text_class_embs.append(average_with_harmonic_series(sentence_model.encode([p[1].raw_text for p in class_map[c_id]])))

full_text_sim = cosine_similarity_embeddings(sentence_model.encode([collection[p_id].raw_text for p_id in target_node.papers]), full_text_class_embs)
full_text_preds = full_text_sim.argmax(axis=1)
full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

# for idx, paper_id in enumerate(target_node.papers):
#     target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

before ranking-based classification:  0.8194444444444444


100%|██████████| 3/3 [00:01<00:00,  2.66it/s]


ranking-based classification 0.9861111111111112


In [ ]:
title: 0.24444444444444444
title + abstract: 0.26666666666666666
raw_text: 0.28888888888888886

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"

In [14]:
device = torch.device("cuda")

In [15]:
tokenizer = BertTokenizer.from_pretrained('/home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/')
model = BertModel.from_pretrained('/home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/', output_hidden_states=True).to(device)

Some weights of BertModel were not initialized from the model checkpoint at /home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [310]:
cnt = defaultdict(int)
all_raw_phrases = []
token_lens = {}

for paper in tqdm(collection):
	all_raw_phrases.extend(paper.raw_text.strip().split())

for phrase in tqdm(set(all_raw_phrases)):
	token_lens[phrase] = len(tokenizer(phrase).input_ids)

cnt = Counter(all_raw_phrases)

100%|██████████| 51136/51136 [00:06<00:00, 7733.38it/s]


In [253]:
def computeProp(phrase_sim, sent_sim):
    print('phrase avg:', phrase_sim.mean(axis=0))
    print('sent avg:', sent_sim.mean(axis=0))
    phrase_count = Counter(phrase_sim.argmax(axis=1))
    phrase_list = np.array([phrase_count[i] 
                            if i in phrase_count 
                            else 0 
                            for i in np.arange(len(taxo.root.children))])
    sent_count = Counter(sent_sim.argmax(axis=1))
    sent_list = np.array([sent_count[i] 
                          if i in sent_count 
                          else 0 
                          for i in np.arange(len(taxo.root.children))])
    
    print(f"Phrase: {phrase_list/sum(phrase_list)}")
    print(f"Sentence: {sent_list/sum(sent_list)}")
    return phrase_count, sent_count

In [352]:
def chunkify(text, token_lens, length=512):
	chunks = [[]]
	split_text = text.split()
	count = 0
	for word in split_text:
		new_count = count + token_lens[word] + 2 # 2 for [CLS] and [SEP]
		if new_count > length:
			chunks.append([word])
			count = token_lens[word]
		else:
			chunks[len(chunks) - 1].append(word)
			count = new_count
	
	chunks = [" ".join(chunk) for chunk in chunks]
	return chunks if len(chunks) > 1 else chunks[0]

def encode(w):
	if type(w) == str:
		chunks = chunkify(w, token_lens)
	else:
		chunks = w
	
	encoded_data = tokenizer(chunks, padding=True, return_tensors="pt").to(device)
	with torch.no_grad():
		output = model(**encoded_data)
	
	# Get all hidden states
	if (type(w) == str) and (len(output.last_hidden_state.shape) == 3):
		return output.last_hidden_state.mean(axis=(0,1)).cpu().numpy()
	else:
		return output.last_hidden_state.mean(axis=1).cpu().numpy()

In [106]:
cnt = defaultdict(int)
vocab_emb = {}
vocab_sent_emb = {}

for paper in tqdm(collection):
	data = paper.raw_text.strip().split()
	sent_data = paper.raw_text.strip().split(" . ")
	p_embs = sentence_model.encode(data)
	s_embs = sentence_model.encode(sent_data)
	for w_id, word in enumerate(data):
		cnt[word] += 1
		if word not in vocab_emb:
			vocab_emb[word] = p_embs[w_id]
	for s_id, sent in enumerate(sent_data):
		vocab_sent_emb[sent] = s_embs[s_id]


100%|██████████| 72/72 [05:51<00:00,  4.88s/it]


In [93]:
for w in tqdm(all_common_phrases):
    if w not in vocab_emb:
        vocab_emb[w] = sentence_model.encode(w)

100%|██████████| 423/423 [00:03<00:00, 135.99it/s]


In [100]:
sorted(cnt.items(), key=lambda x: x[1], reverse=True)

[(',', 46604),
 ('.', 31280),
 ('the', 30334),
 ('and', 14675),
 (')', 14552),
 ('(', 14263),
 ('of', 12858),
 ('to', 10450),
 ('in', 10030),
 ('a', 9576),
 ('for', 6881),
 (':', 6764),
 ('we', 6445),
 ('is', 5707),
 ('on', 5222),
 ('as', 4366),
 ('with', 4331),
 ('[', 3910),
 ('graph', 3906),
 (']', 3898),
 ('et', 3473),
 ('that', 3272),
 ('are', 3188),
 (';', 2906),
 ('model', 2896),
 ('al.', 2762),
 ('from', 2608),
 ('this', 2602),
 ('by', 2589),
 ('can', 2191),
 ('1', 1978),
 ('our', 1948),
 ('be', 1889),
 ('%', 1851),
 ('tasks', 1850),
 ('models', 1797),
 ('llms', 1727),
 ('which', 1720),
 ('an', 1640),
 ('text', 1515),
 ('2', 1514),
 ('performance', 1494),
 ('it', 1456),
 ('reasoning', 1379),
 ('information', 1368),
 ('node', 1330),
 ('molecule', 1328),
 ('data', 1311),
 ('nodes', 1249),
 ('language', 1242),
 ('task', 1206),
 ('3', 1181),
 ('each', 1145),
 ('or', 1129),
 ('table', 1107),
 ('results', 1087),
 ('have', 1038),
 ('not', 1023),
 ('dataset', 1015),
 ('paper', 1003),
 (

In [102]:
iv_phrases = [w for w in cnt if (w not in stopwords.words('english'))]
phrase_reprs = np.concatenate([vocab_emb[w].reshape((-1, 768)) for w in iv_phrases], axis=0)

In [108]:
sents = list(vocab_sent_emb.keys())
sent_reprs = np.concatenate([vocab_sent_emb[s].reshape((-1, 768)) for s in sents], axis=0)

In [60]:
if not os.path.exists(f"SeeTopic/{args.dataset}"):
    os.makedirs(f"SeeTopic/{args.dataset}")

with open(f"SeeTopic/{args.dataset}/{args.dataset}.txt", "w") as f:
    for p in taxo.external_collection:
        f.write(f"paper_title : {p.title} ; paper_abstract : {p.abstract}\n")
    for p in taxo.collection:
        f.write(f"paper_title : {p.title} ; paper_abstract : {p.abstract}\n")

children_with_terms = taxo.root.getChildren(terms=True)

with open(f"SeeTopic/{args.dataset}/keywords_0.txt", "w") as f:
    for idx, c in enumerate(children_with_terms):
        str_c = ",".join(c[1])
        f.write(f"{idx}:{c[0]},{str_c}\n")

In [61]:
os.chdir("./SeeTopic")
subprocess.check_call(['./seetopic.sh', args.dataset, str(args.iters), args.model])
os.chdir("../")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


===Get PLM Embeddings===


Some weights of BertModel were not initialized from the model checkpoint at /home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


####### CONSTRUCTING AND TOKENIZING VOCAB #######
####### COMPUTING STATIC EMBEDDINGS #######
####### COMPUTING RAW WORD EMBEDDINGS #######


100%|██████████| 116/116 [00:30<00:00,  3.79it/s]


===Iter 0: PLM Module===
===Iter 1: PLM Module===
===Iter 1: Local Module===
make: 'cate' is up to date.
Starting training using file ../llm_graph/llm_graph.txt
Reading topics from file llm_graph_1/keywords.txt
Vocab size: 14396
Words in train file: 1290798
Read 3 topics
graph_context	representation_learning	graph_search	
augmentation_methods	alignment_methods	graph_neural_networks	
representation_enhancement	sequence_graph	graph_to_sequence	
Pre-training for 2 epochs, in total 2 + 10 = 12 epochs
Alpha: 0.000099  Progress: 99.67%  Words/thread/sec: 13.26k  Topic mining results written to file llm_graph_1/res_cate.txt
===Iter 1: Ensemble===
===Iter 2: PLM Module===
===Iter 2: Local Module===
make: 'cate' is up to date.
Starting training using file ../llm_graph/llm_graph.txt
Reading topics from file llm_graph_2/keywords.txt
Vocab size: 14396
Words in train file: 1290798
Read 3 topics
graph_context	representation_learning	graph_search	graph_reasoning	answer_generation	graph_generation	
au